# Snake Multiplayer (Client)

In [100]:
%serialconnect --port=/dev/cu.usbserial-8 --baud=115200 

import machine
import display

display = display.Display()

Connecting to --port=/dev/cu.usbserial-8 --baud=115200 
Ready.


In [101]:
from micropython import const

import uasyncio as asyncio
import aioble
from bluetooth import UUID

import random
import struct
import json

class BleClient:
    SERVER_NAME = "SnakeMultiplayer"

    SNAKE_SERVICE_UUID = UUID("ca07faee-7e95-4856-b44b-bbaef52ec7b4")
    
    APPLE_POSITION_CHARACTERISTIC_UUID = UUID("cec23f9f-cdc0-4577-8798-8dd4b01724d8")
    SNAKE1_POSITIONS_CHARACTERISTIC_UUID = UUID("e72c988f-7279-4b82-b808-42884a4ba48f")
    SNAKE2_POSITIONS_CHARACTERISTIC_UUID = UUID("1673be03-ef60-4fb9-869c-4513a8e212f1")
    SNAKE2_DIRECTIONS_CHARACTERISTIC_UUID = UUID("1533130f-7251-4a87-a976-1b5bc0fae798")

    transmittedSnake1Tails = []
    transmittedSnake2Tails = []
    apple_position = None

    def __init__(self) -> None:
        self.snakeServiceConnection = None
        self.snake1TailsCharacteristic = None
        self.snake2TailsCharacteristic = None
        self.snake2DirectionCharacteristic = None
        self.applePositionCharacteristic = None

    async def readSnake1Tails(self):
        value = await self.snake1TailsCharacteristic.notified()
        if (value):
            data = list(json.loads(value))
            snakeLength = int(data[0])
            newSnakeTails = data[1]
            for newTail in newSnakeTails:
                self.transmittedSnake1Tails.append(newTail)
            while len(self.transmittedSnake1Tails) > snakeLength:
                self.transmittedSnake1Tails.pop(0)

        return self.transmittedSnake1Tails
    
    async def readSnake2Tails(self):
        value = await self.snake2TailsCharacteristic.notified()
        if (value):
            data = list(json.loads(value))
            snakeLength = data[0]
            newSnakeTails = data[1]

            for i in range(0, len(newSnakeTails)):
                self.transmittedSnake2Tails.append(newSnakeTails[i])

            while len(self.transmittedSnake2Tails) > snakeLength:
                self.transmittedSnake2Tails.pop(0)

        return self.transmittedSnake2Tails
    
    async def readApplePosistion(self):
        value = await self.applePositionCharacteristic.notified()
        if (value):
            newApplePos = list(json.loads(value))
            self.applePosition = newApplePos

        return self.applePosition
    
    async def writeSnake2Direction(self, newDirection: int):
        data = struct.pack("<h", newDirection)
        await self.snake2DirectionCharacteristic.write(data)
        print("Send new direction " + str(newDirection))
        
    async def findServer(self):
        if (self.snakeServiceConnection is not None):
            return True
        
        async with aioble.scan(5000, interval_us=30000, window_us=30000, active=True) as scanner:
            async for result in scanner:
                if result.name() == self.SERVER_NAME and self.SNAKE_SERVICE_UUID in result.services():
                    print("It's a match")
                    device = result.device
                    connection = await device.connect()
                    self.snakeServiceConnection = await connection.service(self.SNAKE_SERVICE_UUID)
                    self.snake1TailsCharacteristic = await self.snakeServiceConnection.characteristic(self.SNAKE1_POSITIONS_CHARACTERISTIC_UUID)
                    await self.snake1TailsCharacteristic.read()
                    await self.snake1TailsCharacteristic.subscribe(notify=True, indicate=False)
                    self.snake2TailsCharacteristic = await self.snakeServiceConnection.characteristic(self.SNAKE2_POSITIONS_CHARACTERISTIC_UUID)
                    await self.snake2TailsCharacteristic.subscribe(notify=True, indicate=False)
                    self.applePositionCharacteristic = await self.snakeServiceConnection.characteristic(self.APPLE_POSITION_CHARACTERISTIC_UUID)
                    await self.applePositionCharacteristic.subscribe(notify=True, indicate=False)                    
                    self.snake2DirectionCharacteristic = await self.snakeServiceConnection.characteristic(self.SNAKE2_DIRECTIONS_CHARACTERISTIC_UUID)
                    return True
            
        return False

In [102]:
BLOCK_SIZE = 4

DISPLAY_WIDTH = 128
DISPLAY_HEIGHT = 64

GAME_WIDTH = int(DISPLAY_WIDTH / BLOCK_SIZE)
GAME_HEIGHT = int(DISPLAY_HEIGHT / BLOCK_SIZE)

def drawScore(score):
    display.text("Score " + str(score), 0, 0)

def drawGame(snake1, snake2, apple):
    display.clear()
    
    for s1 in snake1:
        drawPoint(s1[0], s1[1])
    
    for s2 in snake2:
        drawPoint(s2[0], s2[1])

    drawPoint(apple[0], apple[1])

    drawScore(int((len(snake2)-2) / 2))

    display.show()



def drawPoint(x, y):
    display.fill_rect(x * BLOCK_SIZE, y * BLOCK_SIZE, BLOCK_SIZE, BLOCK_SIZE)

In [103]:
from time import sleep
bleClient = BleClient()

downBtn = Pin(14, Pin.IN, Pin.PULL_UP)
rightBtn = Pin(21, Pin.IN, Pin.PULL_UP)
leftBtn = Pin(18, Pin.IN, Pin.PULL_UP)
upBtn = Pin(25, Pin.IN, Pin.PULL_UP)


async def main():
    global bleClient, drawGame
    oldDirection = 1
    direction = 1

    while True:
        result = await bleClient.findServer()
        if result is False:
            continue

        if downBtn.value() == 0:
            direction = 1
        if rightBtn.value() == 0:
            direction = 3
        if leftBtn.value() == 0:
            direction = 2
        if upBtn.value() == 0:
            direction = 0

        if direction != oldDirection:
            await bleClient.writeSnake2Direction(direction)
            oldDirection = direction
    

        snake1Tails = await bleClient.readSnake1Tails()
        snake2Tails = await bleClient.readSnake2Tails()
        applePos = await bleClient.readApplePosistion()

    
        drawGame(snake1Tails, snake2Tails, applePos)

        sleep(0.02)

loop = asyncio.get_event_loop()
loop.run_until_complete(main())

It's a match
.Send new direction 0
Send new direction 2
Send new direction 3
Send new direction 1
Send new direction 3
.Send new direction 0
....